In [1]:
# coding:utf-8
import os
from bs4 import BeautifulSoup
import pandas as pd
import re
import zhconv

# 准备文本数据总数组，里面每个数组都是一个样例中的文本内容
data_list = []
# 读取文件
# file_data来源于2017中国网络安全技术对抗赛《恶意网页分析》赛题的样本数据集
filedir = r'file_data/恶意网页分析/file1'
filename = os.listdir(filedir)

# 导入file_Dataframe
file_df = pd.read_csv(r"file_url.csv")
file_df['text'] = ''
err = 0
# 扫描文件列表，处理列表中的文件，提取文件中的html中的纯文本（只保留汉字）， 将文本拼接到file_url.csv文件中形成分词前的数据
# 这里的file_url.csv是从file文件中提取出来的url及其标签等信息。
for i in range(len(filename)):
    sub_filename = filedir+'/'+filename[i]
    # 找到filename所在的行
    if file_df.index[file_df['id'] == filename[i]].size > 0:
        row_index = file_df.index[(file_df['id'] == filename[i])][0]

        try:
            with open(sub_filename, 'r', encoding='utf-8', errors='ignore') as f:
                obj = f.read()
                text = obj
                # 从html中提取文本信息
                soup = BeautifulSoup(text, 'lxml')
                file_list = soup.text.split(" ")
                txt = "".join(file_list)
                # 只保留文本信息中的汉字和字母
                txt = re.sub(r'[^\u4e00-\u9fa5]', '', txt)
                # 繁体字转化成简体字
                txt = zhconv.convert(str(txt), 'zh-hans')
                file_df.loc[row_index, 'pagetxt'] = txt
        except:
            err = err+1
            pass
    # data_list.append(file_list)
print(err)
file_df.to_csv("benign.csv", encoding="utf_8_sig")

C:\Anaconda\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


0


In [2]:
df = pd.read_csv(r'benign.csv')

使用哈工大的停用词表以及jieba工具进行词切分

In [3]:
import jieba
import re
def get_word(text, word_dic, stop_words):
  # 将文本切分为单个词
  words = jieba.cut(text)
  for w in words:
    # 构建词表
    if w in stop_words:
      continue
    if w in word_dic:
      word_dic[w] += 1
    else:
      word_dic[w] = 1

In [4]:
def read_stopwords():
  stop_words = []
#这里自己更改文件所在名
  with open(r"hit_stopwords.txt", mode="r", encoding="utf-8") as f:
    for w in f.read().splitlines():
      stop_words.append(w)
  return stop_words

In [5]:
# 生成词表字典
corpus = []
stop_words = read_stopwords()
word_dic = {}
df['text'] = df['pagetxt'].apply(lambda i : get_word(str(i), word_dic, stop_words))
word_list = sorted(word_dic.items(), key=lambda item: item[1], reverse=True)
print(type(word_list))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Hs\AppData\Local\Temp\jieba.cache
Loading model cost 0.388 seconds.
Prefix dict has been built successfully.


<class 'list'>


In [6]:
with open('benign_word.txt', 'w', encoding='utf-8') as f:
    for word, freq in word_list:
        f.write(word + " " + str(freq) + "\n")